In [3]:
#!pip install implicit

In [4]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [7]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [8]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [26]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [27]:
%%time
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

weighted_items = data.groupby('item_id')['sales_value'].sum().reset_index()
sales_sum = weighted_items['sales_value'].sum()
weighted_items['weight'] = weighted_items['sales_value'] / sales_sum

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(weighted_items, n=5))

CPU times: user 3.73 s, sys: 81 ms, total: 3.81 s
Wall time: 3.8 s


In [28]:
result.head(5)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13777408, 945294, 962229, 5569471, 985622]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1027284, 5585635, 5570383, 6534178, 6442594]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1077827, 844165, 894401, 9419851, 1106523]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[6551861, 5569303, 5569471, 956995, 6534178]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1004411, 926916, 5568447, 871061, 992128]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [30]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[553111, 990130, 14025228, 8091299, 855288]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[8155825, 1841061, 946817, 1541203, 8203466]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [54]:
import re

In [71]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(list(map(int, re.findall(r'\d+', bought_list))))
    recommended_list = np.array(list(map(int, re.findall(r'\d+', recommended_list))))[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision

In [72]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0004:random_recommendation
0.1552:popular_recommendation
0.1369:itemitem
0.1329:cosine
0.139:tfidf
0.2019:own_purchases


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


Лучшее качество показывае алгоритм own_purchases

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code